In [10]:
import habitat_sim
import random
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import quaternion
import os

In [11]:
# path_to_mp3d = "path/to/habitat-api/data/scene_datasets/mp3d/"
# folder_content = os.listdir(path_to_mp3d)
# scene_paths = []
# for content in folder_content:
#     full_path = path_to_mp3d + content
#     if os.path.isdir(full_path):
#         full_path = full_path + "/" + content + ".glb"
#         # print(full_path)
#         scene_paths.append(full_path)
        
# print("Total number of scenes:", len(scene_paths))
# test_scene = scene_paths[1]
# print("Scene to be used:", test_scene)

modified_scene = "/home/cengerkin/Desktop/habitat-full/habitat-api/data/scene_datasets/mp3d/testing/new/exp/exp1_semantic.ply"

sim_settings = {
    "width": 256,  # Spatial resolution of the observations    
    "height": 256,
    "scene": modified_scene,  # Scene path
    "default_agent": 0,  
    "sensor_height": 0,  # Height of sensors in meters
    "color_sensor": True,  # RGB sensor
    "semantic_sensor": True,  # Semantic sensor
    "depth_sensor": True,  # Depth sensor
    "seed": 1,
}

# orig_scene = "/home/cengerkin/Desktop/habitat-full/habitat-api/data/scene_datasets/mp3d/testing/new/region0_orig_semantic.ply"

# sim_settings = {
#     "width": 256,  # Spatial resolution of the observations    
#     "height": 256,
#     "scene": orig_scene,  # Scene path
#     "default_agent": 0,  
#     "sensor_height": 0,  # Height of sensors in meters
#     "color_sensor": True,  # RGB sensor
#     "semantic_sensor": True,  # Semantic sensor
#     "depth_sensor": True,  # Depth sensor
#     "seed": 1,
# }

# Simulator config

In [4]:
def make_cfg(settings):
    sim_cfg = habitat_sim.SimulatorConfiguration()
    sim_cfg.gpu_device_id = 0
    sim_cfg.scene.id = settings["scene"]
    
    # Note: all sensors must have the same resolution
    sensors = {
        "color_sensor": {
            "sensor_type": habitat_sim.SensorType.COLOR,
            "resolution": [settings["height"], settings["width"]],
            "position": [0.0, settings["sensor_height"], 0.0],
        },
        "depth_sensor": {
            "sensor_type": habitat_sim.SensorType.DEPTH,
            "resolution": [settings["height"], settings["width"]],
            "position": [0.0, settings["sensor_height"], 0.0],
        },
        "semantic_sensor": {
            "sensor_type": habitat_sim.SensorType.SEMANTIC,
            "resolution": [settings["height"], settings["width"]],
            "position": [0.0, settings["sensor_height"], 0.0],
        },  
    }
    
    sensor_specs = []
    for sensor_uuid, sensor_params in sensors.items():
        if settings[sensor_uuid]:
            sensor_spec = habitat_sim.SensorSpec()
            sensor_spec.uuid = sensor_uuid
            sensor_spec.sensor_type = sensor_params["sensor_type"]
            sensor_spec.resolution = sensor_params["resolution"]
            sensor_spec.position = sensor_params["position"]

            sensor_specs.append(sensor_spec)
            
    # Here you can specify the amount of displacement in a forward action and the turn angle
    agent_cfg = habitat_sim.agent.AgentConfiguration()
    agent_cfg.sensor_specifications = sensor_specs
    agent_cfg.action_space = {
        "move_forward": habitat_sim.agent.ActionSpec(
            "move_forward", habitat_sim.agent.ActuationSpec(amount=0.25)
        ),
        "move_backward": habitat_sim.agent.ActionSpec(
            "move_backward", habitat_sim.agent.ActuationSpec(amount=0.25)
        ),
        "move_right": habitat_sim.agent.ActionSpec(
            "move_right", habitat_sim.agent.ActuationSpec(amount=0.25)
        ),
        "move_left": habitat_sim.agent.ActionSpec(
            "move_left", habitat_sim.agent.ActuationSpec(amount=0.25)
        ),
        "turn_left": habitat_sim.agent.ActionSpec(
            "turn_left", habitat_sim.agent.ActuationSpec(amount=15.0)
        ),
        "turn_right": habitat_sim.agent.ActionSpec(
            "turn_right", habitat_sim.agent.ActuationSpec(amount=15.0)
        ),
        "look_up": habitat_sim.agent.ActionSpec(
            "look_up", habitat_sim.agent.ActuationSpec(amount=1)
        ),
        "look_down": habitat_sim.agent.ActionSpec(
            "look_down", habitat_sim.agent.ActuationSpec(amount=1)
        ),
    }
    
    return habitat_sim.Configuration(sim_cfg, [agent_cfg])

cfg = make_cfg(sim_settings)
sim = habitat_sim.Simulator(cfg)

W0807 22:46:10.247082 16726 simulator.py:167] Could not find navmesh /home/cengerkin/Desktop/habitat-full/habitat-api/data/scene_datasets/mp3d/testing/new/exp/exp1_semantic.navmesh, no collision checking will be done


# Scene semantic annotations

In [5]:
def print_scene_recur(scene, limit_output=10):
    print(f"House has {len(scene.levels)} levels, {len(scene.regions)} regions and {len(scene.objects)} objects")
    print(f"House center:{scene.aabb.center} dims:{scene.aabb.sizes}")
    
    count = 0
    for level in scene.levels:
        print(
            f"Level id:{level.id}, center:{level.aabb.center},"
            f" dims:{level.aabb.sizes}"
        )
        for region in level.regions:
            print(
                f"Region id:{region.id}, category:{region.category.name()},"
                f" center:{region.aabb.center}, dims:{region.aabb.sizes}"
            )
            for obj in region.objects:
                print(
                    f"Object id:{obj.id}, category:{obj.category.name()},"
                    f" center:{obj.aabb.center}, dims:{obj.aabb.sizes}"
                )
                count += 1
                if count >= limit_output:
                    return None

# Print semantic annotation information (id, category, bounding box details) 
# about levels, regions and objects in a hierarchical fashion
scene = sim.semantic_scene
print_scene_recur(scene)

House has 0 levels, 0 regions and 0 objects
House center:[0. 0. 0.] dims:[-inf -inf -inf]


In [6]:
random.seed(sim_settings["seed"])
sim.seed(sim_settings["seed"])

# Set agent state
agent = sim.initialize_agent(sim_settings["default_agent"])
agent_state = habitat_sim.AgentState()
agent_state.position = np.array([-2.6470983, 4.6513968, 6.468966]) # TODO: Change here as desired
agent_state.rotation = quaternion.quaternion(0.258819073438644, 0, -0.965925872325897, 0) # TODO: Change here as desired
agent.set_state(agent_state)

# Get agent state
agent_state = agent.get_state()
print("agent_state: position", agent_state.position, "rotation", agent_state.rotation)

agent_state: position [-2.6470983  4.6513968  6.468966 ] rotation quaternion(0.258819073438644, 0, -0.965925872325897, 0)


In [12]:
from PIL import Image
from habitat_sim.utils.common import d3_40_colors_rgb

def display_sample(rgb_obs, semantic_obs, depth_obs):
    """Plot RGB, Semantic and Depth images"""
    rgb_img = Image.fromarray(rgb_obs, mode="RGB")
    
    semantic_img = Image.new("P", (semantic_obs.shape[1], semantic_obs.shape[0]))
    semantic_img.putpalette(d3_40_colors_rgb.flatten())
    semantic_img.putdata((semantic_obs.flatten() % 40).astype(np.uint8))
    semantic_img = semantic_img.convert("RGB")
    
    depth_img = Image.fromarray((depth_obs / 10 * 255).astype(np.uint8), mode="L")

    arr = [rgb_img, semantic_img, depth_img]
    titles = ['rgb', 'semantic', 'depth']
    plt.figure(figsize=(12 ,8))
    for i, data in enumerate(arr):
        ax = plt.subplot(1, 3, i+1)
        ax.axis('off')
        ax.set_title(titles[i])
        plt.imshow(data)
    plt.show()
    
def get_camera_matrices(position, rotation):
    rotation = quaternion.as_rotation_matrix(rotation)
    
    # Pinv: Agent/Camera pose wrt Habitat WCS
    Pinv = np.eye(4)
    Pinv[0:3, 0:3] = rotation
    Pinv[0:3, 3] = position
    # P: Habitat WCS wrt Agent/Camera
    P = np.linalg.inv(Pinv)

    return P, Pinv

def get_visuals(observations):
    """Returns PIL versions of RGB, Semantic and Depth images, also returns Depth array"""
    rgb_img = observations["color_sensor"]
    rgb_img = Image.fromarray(rgb_img, mode="RGB")
    
    sem = observations["semantic_sensor"]
    sem_img = Image.new("P", (sem.shape[1], sem.shape[0]))
    sem_img.putpalette(d3_40_colors_rgb.flatten())
    sem_img.putdata((sem.flatten() % 40).astype(np.uint8))
    sem_img = sem_img.convert("RGB")
    
    dep_arr = observations["depth_sensor"]
    dep_img = Image.fromarray((dep_arr / 10 * 255).astype(np.uint8), mode="L")
    
    return rgb_img, sem_img, dep_img, dep_arr

def collect_all_data(observations, state):
    rgb_img, sem_img, _, dep_arr = get_visuals(observations)
    P, Pinv = get_camera_matrices(state.position, state.rotation)
    return rgb_img, sem_img, dep_arr, Pinv

def split_RT(RT):
    formatter={'float_kind':lambda x: "%.10f" % x}
    R = RT[0:3, 0:3]
    cam_pos = RT[0:3, 3].ravel()
    cam_up = R[:, 1].ravel()  # y=cam_up (already unit)
    cam_dir = R[:, 2].ravel() # z=cam_dir (already unit)
    cam_pos = np.array2string(cam_pos, formatter=formatter, max_line_width=np.inf, separator=", ")
    cam_up = np.array2string(cam_up, formatter=formatter, max_line_width=np.inf, separator=", ")
    cam_dir = np.array2string(cam_dir, formatter=formatter, max_line_width=np.inf, separator=", ")
    return cam_pos, cam_up, cam_dir

def save_data(path, index, rgb, sem, dep, Pinv):
    file_name = "sample_" + str(index)
    rgb.save(os.path.join(path, file_name + ".png"))
    sem.save(os.path.join(path, file_name + ".seg.png"))
    np.save(os.path.join(path, file_name + ".depth.npy"), dep)
    
    cam_file_content = "{:<12} = {}';\n"
    cam_pos, cam_up, cam_dir = split_RT(Pinv)
    info = cam_file_content.format("cam_pos", cam_pos)
    info += cam_file_content.format("cam_dir", cam_dir)
    info += cam_file_content.format("cam_up", cam_up)
    with open(os.path.join(path, file_name + ".txt"), 'w+') as f:
        f.write(info)

# GUI with PyQt5

In [8]:
PATH1 = "/home/cengerkin/Desktop/habitat-full/habitat-api/data/scene_datasets/mp3d/testing/new/exp/modified"
PATH2 = "/home/cengerkin/Desktop/habitat-full/habitat-api/data/scene_datasets/mp3d/testing/new/exp"
PATH3 = "/home/cengerkin/Desktop/habitat-full/habitat-api/data/scene_datasets/mp3d/testing/new/exp/orig"

In [9]:
import sys

from PyQt5.QtWidgets import *
from PyQt5 import QtGui
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import Qt

from PIL.ImageQt import ImageQt

action_names = list(
    cfg.agents[
        sim_settings["default_agent"]
    ].action_space.keys()
)

action_map = {
    Qt.Key_4: "turn_left",
    Qt.Key_6: "turn_right",
    Qt.Key_8: "look_up",
    Qt.Key_5: "look_down",
    Qt.Key_W: "move_forward",
    Qt.Key_A: "move_left",
    Qt.Key_S: "move_backward",
    Qt.Key_D: "move_right"
}

state_hist = [] # Keep agent state when a sample is taken

class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.initialize()
        
    def get_imageQt(self, observations):
        """Returns Qt versions of RGB, Semantic and Depth images"""
        rgb_img, sem_img, dep_img, _ = get_visuals(observations)
        rgb_img = ImageQt(rgb_img)
        rgb_img = QPixmap.fromImage(rgb_img)
        
        sem_img = ImageQt(sem_img)
        sem_img = QPixmap.fromImage(sem_img)
        
        dep_img = ImageQt(dep_img)
        dep_img = QPixmap.fromImage(dep_img)
        return rgb_img, sem_img, dep_img 
        
    def initialize(self):
        self.title = "Habitat Agent"
        self.top = 0
        self.left = 0
        self.width = 256*3
        self.height = 456
        self.timestep = 0
        
        hbox = QHBoxLayout()
        
        rgb_panel = QFrame()
        rgb_panel.setFrameShape(QFrame.StyledPanel)
        self.rgb_panel = QLabel(rgb_panel)
        
        seg_panel = QFrame()
        seg_panel.setFrameShape(QFrame.StyledPanel)
        self.seg_panel = QLabel(seg_panel)
        
        dep_panel = QFrame()
        dep_panel.setFrameShape(QFrame.StyledPanel)
        self.dep_panel = QLabel(dep_panel)
        
        self.info_panel = info_panel = QPlainTextEdit()
        info_panel.setReadOnly(True)

        split1 = QSplitter(Qt.Horizontal)
        split1.addWidget(rgb_panel)
        split1.addWidget(seg_panel)
        split1.setSizes([256,256])
        
        split2 = QSplitter(Qt.Horizontal)
        split2.addWidget(split1)
        split2.addWidget(dep_panel)
        split2.setSizes([512,256])
        
        split3 = QSplitter(Qt.Vertical)
        split3.addWidget(split2)
        split3.addWidget(info_panel)
        split3.setSizes([256,200])
        hbox.addWidget(split3)
        
        # Render images on respective windows
        observations = sim.get_sensor_observations()
        agent_state = agent.get_state()
        # P, Pinv = get_camera_matrices(agent_state.position, agent_state.rotation)
        
        rgb, seg, dep = self.get_imageQt(observations)
        self.rgb_panel.setPixmap(rgb)
        self.seg_panel.setPixmap(seg)
        self.dep_panel.setPixmap(dep)

        log = "t: {}, Position: {}, Orientation: {}".format(self.timestep, agent_state.position, agent_state.rotation)
        self.info_panel.appendPlainText(log)
        
        self.setLayout(hbox)
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)
        
        self.show()

    def keyPressEvent(self, event):
        key = event.key()
        # Clear logger
        if key == Qt.Key_C:
            self.info_panel.clear()
        
        # Close window
        elif key == Qt.Key_Escape:
            self.close()
        
        # Save current observation
        elif key == Qt.Key_P:
            observations = sim.get_sensor_observations()
            agent_state = agent.get_state()
            file_index = len(state_hist)
            state_hist.append(agent_state)
            data = collect_all_data(observations, agent_state)
            save_data(PATH1, file_index, *data)
            log = "Saving data at t:{}".format(self.timestep)
            self.info_panel.appendPlainText(log)
            
        # TODO: Adjustable speed
        elif key == Qt.Key_Plus:
            agent.agent_config.action_space["move_forward"].actuation.amount += 0.1
        elif key == Qt.Key_Minus:
            agent.agent_config.action_space["move_forward"].actuation.amount -= 0.1
        
        # Take an action
        else:
            action = action_map[key]
            observations = sim.step(action)
            self.timestep += 1
            
            agent_state = agent.get_state()
            # P, Pinv = get_camera_matrices(agent_state.position, agent_state.rotation)
            
            rgb, seg, dep = self.get_imageQt(observations)
            self.rgb_panel.setPixmap(rgb)
            self.seg_panel.setPixmap(seg)
            self.dep_panel.setPixmap(dep)
            
            log = "t:{}, Position: {}, Orientation: {}".format(self.timestep, agent_state.position, agent_state.rotation)
            self.info_panel.appendPlainText(log)



app = QApplication([])

window = MainWindow()

window.show()
app.exec_()

0

In [17]:
# Identify segmentation color for the modified object
OBJID = 12
color = d3_40_colors_rgb[OBJID]
print(color)
color = np.broadcast_to(color, (256, 256, 3))
img = Image.fromarray(color, mode="RGB")
img.show()

[227 119 194]


# Sample from the Same Trajectory

In [ ]:
# Save trajectory
line = "Position: {}, Orientation: {}\n"

with open(os.path.join(PATH2, "trajectory.txt"), "w+") as file:
    for state in state_hist:
        newline = line.format(state.position, state.rotation)
        file.write(newline)

In [ ]:
# Open unmodified scene and sample images from the same sequence.

agent_state = agent.get_state()

for time, state in enumerate(state_hist):
    agent_state.position = state.position
    agent_state.rotation = state.rotation
    observations = sim.get_sensor_observations()
    data = collect_all_data(observations, agent_state)
    save_data(PATH3, time, *data)

# Take Specified Actions

In [ ]:
from IPython.display import clear_output
action_names = list(
    cfg.agents[
        sim_settings["default_agent"]
    ].action_space.keys()
)

action_map = {
    49: "turn_left",
    50: "turn_right",
    51: "look_up",
    52: "look_down",
    **dict.fromkeys((87, 119), "move_forward"),
    **dict.fromkeys((65, 97), "move_left"),
    **dict.fromkeys((83, 115), "move_backward"),
    **dict.fromkeys((68, 100), "move_right")
}

run = True

while run:
    try:
        button = ord(input("Press a key [W/A/S/D/1/2/3/4]: "))
        if button not in (69, 99):
            #clear_output(): flush output
            action = action_map[button]
            print("Action", action)
            observations = sim.step(action)
        else:
            observations = sim.get_sensor_observations()
        
        rgb = observations["color_sensor"]
        semantic = observations["semantic_sensor"]
        depth = observations["depth_sensor"]

        display_sample(rgb, semantic, depth)
        
#         agent_state = agent.get_state()
#         to_cam, to_habitat = get_camera_matrices(agent_state.position, agent_state.rotation)
#         print("Agent state, Position:", agent_state.position, "Rotation:", agent_state.rotation)
#         print("From Habitat to Camera:")
#         print(to_cam)
#         print("From Camera Matrix to Habitat:")
#         print(to_habitat)

    except Exception as e:
        print(e)
        run = False

# Take Random Actions and Display Sensor Data

In [ ]:
total_frames = 0
action_names = list(
    cfg.agents[
        sim_settings["default_agent"]
    ].action_space.keys()
)

print(action_names)

max_frames = 5

while total_frames < max_frames:
    action = random.choice(action_names)
    print("action", action)
    observations = sim.step(action)
    rgb = observations["color_sensor"]
    semantic = observations["semantic_sensor"]
    depth = observations["depth_sensor"]
    
    display_sample(rgb, semantic, depth)
    
    total_frames += 1